In [ ]:
"""This is a minimal example of using Tianshou with MARL to train agents.

Author: Will (https://github.com/WillDudley)

Python version used: 3.8.10

Requirements:
pettingzoo == 1.22.0
git+https://github.com/thu-ml/tianshou
"""

import os
from typing import Optional, Tuple

import gymnasium as gym
import numpy as np
import torch
from tianshou.data import Collector, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager, RandomPolicy
from tianshou.trainer import offpolicy_trainer
from tianshou.utils.net.common import Net

from pettingzoo.classic import tictactoe_v3


def _get_agents(
    agent_learn: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    env = _get_env()
    observation_space = (
        env.observation_space["observation"]
        if isinstance(env.observation_space, gym.spaces.Dict)
        else env.observation_space
    )
    if agent_learn is None:
        # model
        net = Net(
            state_shape=observation_space.shape or observation_space.n,
            action_shape=env.action_space.shape or env.action_space.n,
            hidden_sizes=[128, 128, 128, 128],
            device="cuda" if torch.cuda.is_available() else "cpu",
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=1e-4)
        agent_learn = DQNPolicy(
            model=net,
            optim=optim,
            discount_factor=0.9,
            estimation_step=3,
            target_update_freq=320,
        )

    if agent_opponent is None:
        agent_opponent = RandomPolicy()

    agents = [agent_opponent, agent_learn]
    policy = MultiAgentPolicyManager(agents, env)
    return policy, optim, env.agents


def _get_env():
    """This function is needed to provide callables for DummyVectorEnv."""
    return PettingZooEnv(tictactoe_v3.env())


if __name__ == "__main__":
    # ======== Step 1: Environment setup =========
    train_envs = DummyVectorEnv([_get_env for _ in range(10)])
    test_envs = DummyVectorEnv([_get_env for _ in range(10)])

    # seed
    seed = 1
    np.random.seed(seed)
    torch.manual_seed(seed)
    train_envs.seed(seed)
    test_envs.seed(seed)

    # ======== Step 2: Agent setup =========
    policy, optim, agents = _get_agents()

    # ======== Step 3: Collector setup =========
    train_collector = Collector(
        policy,
        train_envs,
        VectorReplayBuffer(20_000, len(train_envs)),
        exploration_noise=True,
    )
    test_collector = Collector(policy, test_envs, exploration_noise=True)
    # policy.set_eps(1)
    train_collector.collect(n_step=64 * 10)  # batch size * training_num

    # ======== Step 4: Callback functions setup =========
    def save_best_fn(policy):
        model_save_path = os.path.join("log", "rps", "dqn", "policy.pth")
        os.makedirs(os.path.join("log", "rps", "dqn"), exist_ok=True)
        torch.save(policy.policies[agents[1]].state_dict(), model_save_path)

    def stop_fn(mean_rewards):
        return mean_rewards >= 0.6

    def train_fn(epoch, env_step):
        policy.policies[agents[1]].set_eps(0.1)

    def test_fn(epoch, env_step):
        policy.policies[agents[1]].set_eps(0.05)

    def reward_metric(rews):
        return rews[:, 1]

    # ======== Step 5: Run the trainer =========
    result = offpolicy_trainer(
        policy=policy,
        train_collector=train_collector,
        test_collector=test_collector,
        max_epoch=50,
        step_per_epoch=1000,
        step_per_collect=50,
        episode_per_test=10,
        batch_size=64,
        train_fn=train_fn,
        test_fn=test_fn,
        stop_fn=stop_fn,
        save_best_fn=save_best_fn,
        update_per_step=0.1,
        test_in_train=False,
        reward_metric=reward_metric,
    )

    # return result, policy.policies[agents[1]]
    print(f"\n==========Result==========\n{result}")
    print("\n(the trained policy can be accessed via policy.policies[agents[1]])")


In [ ]:
from environment import Environment

env = Environment.VehicleJobSchedulingEnvACE()
env.reset()
for agent in env.agent_iter(10000):
    env.step(env.action_space(agent).sample())


In [ ]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random


In [ ]:
key = random.PRNGKey(0)
x = random.normal(key, (10,))
print(x)


In [ ]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()  # runs on the GPU


In [ ]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()


In [ ]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()


In [2]:
from environment import environment_jax

env = environment_jax.VehicleJobSchedulingEnvACE()


AttributeError: module 'jax.numpy' has no attribute 'random'

In [ ]:
from environment import Environment

env = Environment.VehicleJobSchedulingEnvACE()
env.reset()


In [ ]:
env.parameters.cluster.machines[1].observe()


In [7]:
import cProfile
from pettingzoo.test import performance_benchmark
from environment import Environment
for i in range(25,40,5):
    for j in range(10,100,20):
        print("duration: ", i, " average_per_slot: ", j)
        para = Environment.VehicleJobSchedulingParameters(duration=i,average_per_slot=j)
        env = Environment.VehicleJobSchedulingEnvACE()
        performance_benchmark(env)


duration:  25  average_per_slot:  10
Starting performance benchmark
12679.175816391342 turns per second
1056.5979846992784 cycles per second
Finished performance benchmark
duration:  25  average_per_slot:  30
Starting performance benchmark
12731.171350055083 turns per second
1060.9309458379237 cycles per second
Finished performance benchmark
duration:  25  average_per_slot:  50
Starting performance benchmark
12692.710303681068 turns per second
1057.725858640089 cycles per second
Finished performance benchmark
duration:  25  average_per_slot:  70
Starting performance benchmark
12447.760684349141 turns per second
1037.3133903624284 cycles per second
Finished performance benchmark
duration:  25  average_per_slot:  90
Starting performance benchmark
12489.932789005727 turns per second
1040.8277324171438 cycles per second
Finished performance benchmark
duration:  30  average_per_slot:  10
Starting performance benchmark
12766.805595751817 turns per second
1063.9004663126514 cycles per second


In [3]:
import cProfile
from tkinter import E
from pettingzoo.test import performance_benchmark, api_test
from environment import Environment

for i in range(10,100,10):
    para = Environment.VehicleJobSchedulingParameters(average_per_slot=i, duration=30)
    env = Environment.VehicleJobSchedulingEnvACE(parameter=para)
    env.reset()
    for agent in env.agent_iter(100000):
        env.step(env.action_space(agent).sample())   
    print("Finish Rate: ",env.finished_job/env.total_job)


Finish Rate:  0.8738916256157635
Finish Rate:  0.837245696400626
Finish Rate:  0.7435723951285521
Finish Rate:  0.6604767879548307
Finish Rate:  0.5661592505854801
Finish Rate:  0.5196389771017884
Finish Rate:  0.4667235494880546
Finish Rate:  0.43742098609355246
Finish Rate:  0.3821138211382114


In [ ]:
for agent in env.agent_iter(10):
    action = 1
    env.step(action)
    print(action)


In [ ]:
print("env job finished rate", env.finished_job/env.total_job)


In [ ]:
from environment import Environment, AllocationMechanism, Machine

env = Environment.VehicleJobSchedulingEnvACE()
for job in env.get_job_next_step():
    job1 = job
    break
cluster = env.parameters.cluster

fp = AllocationMechanism.FirstPrice()
bids = Machine.Bids(cluster,job1)


In [ ]:
bids.request_bids()
bids.get_bids()
bids.bids


In [ ]:
from tianshou.data import Collector
from tianshou.env import DummyVectorEnv
from tianshou.policy import MultiAgentPolicyManager, RandomPolicy

import tianshou_setup

env = tianshou_setup.get_env()
policies = MultiAgentPolicyManager(
    [
        RandomPolicy(
            observation_space=env.observation_space, action_space=env.action_space
        )
        for _ in range(len(env.agents))
    ],
    env,
)


In [ ]:
pol = RandomPolicy(
            observation_space=env.observation_space, action_space=env.action_space
        )


In [ ]:
pol.forward()


In [ ]:
import time

from tianshou.data import Collector
from tianshou.env import DummyVectorEnv, SubprocVectorEnv
from tianshou.policy import MultiAgentPolicyManager

import tianshou_setup
from static_policy.TruthfulPolicy import TruthfulPolicy

env = tianshou_setup.get_env_continous()


def get_DummyVectorEnv_n(n):
    return DummyVectorEnv([lambda: env for _ in range(n)])


def get_SubprocVectorEnv_n(n):
    return SubprocVectorEnv([lambda: env for _ in range(n)])


def get_policy_manager():
    env = tianshou_setup.get_env_continous()
    policies = MultiAgentPolicyManager(
        [
            TruthfulPolicy(
                observation_space=env.observation_space, action_space=env.action_space
            )
            for _ in range(len(env.agents))
        ],
        env,
    )
    return policies


In [ ]:
n = 10
env = get_DummyVectorEnv_n(n)
policies = get_policy_manager()
collector = Collector(policies, env)


In [ ]:
# Execute one episode
start = time.time()
result = collector.collect(n_episode=10)
end = time.time()
print(f"Time elapsed: {end-start}")


In [ ]:
tp = policies.policies['Machine_0']


In [5]:
def FinishRate(average_per_slot=10,machine_numbers=10):
    from environment import Environment
    para = Environment.VehicleJobSchedulingParameters(average_per_slot=average_per_slot, machine_numbers=machine_numbers)
    env = Environment.VehicleJobSchedulingEnvACE(parameter=para)
    env.reset()
    for agent in env.agent_iter(100000):
        env.step(env.action_space(agent).sample())
    print("Finish Rate: ",env.finished_job/env.total_job)


In [14]:
for average in range(1,40,2):
    print("average: ", average)
    FinishRate(average_per_slot=average,machine_numbers=6)


average:  1
Finish Rate:  0.8653846153846154
average:  3
Finish Rate:  0.9018181818181819
average:  5
Finish Rate:  0.8770161290322581
average:  7
Finish Rate:  0.8705547652916074
average:  9
Finish Rate:  0.84251968503937
average:  11
Finish Rate:  0.8443396226415094
average:  13
Finish Rate:  0.7830609212481426
average:  15
Finish Rate:  0.7724665391969407
average:  17
Finish Rate:  0.7380410022779044
average:  19
Finish Rate:  0.7107085775173149
average:  21
Finish Rate:  0.7013487475915221
average:  23
Finish Rate:  0.6526458616010855
average:  25
Finish Rate:  0.6032372680615871
average:  27
Finish Rate:  0.603460972017673
average:  29
Finish Rate:  0.5652474904811353
average:  31
Finish Rate:  0.5144059566202654
average:  33
Finish Rate:  0.5041398344066237
average:  35
Finish Rate:  0.48461755574372
average:  37
Finish Rate:  0.45884827767551933
average:  39
Finish Rate:  0.43522372528616027


In [16]:
for average in range(2,69,6):
    print("average: ", average)
    FinishRate(average_per_slot=average,machine_numbers=12)


average:  2
Finish Rate:  0.9214659685863874
average:  8
Finish Rate:  0.8761467889908257
average:  14
Finish Rate:  0.8304446119065562
average:  20
Finish Rate:  0.8105749486652978
average:  26
Finish Rate:  0.7671130952380952
average:  32
Finish Rate:  0.7190490892250695
average:  38
Finish Rate:  0.685372340425532
average:  44
Finish Rate:  0.6315431679129844
average:  50
Finish Rate:  0.5862412761714856
average:  56
Finish Rate:  0.5447884227880564
average:  62
Finish Rate:  0.4992874109263658
average:  68
Finish Rate:  0.4751783590963139


In [26]:
for average in range(70,76,1):
    print("average: ", average)
    FinishRate(average_per_slot=average,machine_numbers=24)


average:  70
Finish Rate:  0.6954293037163606
average:  71
Finish Rate:  0.6863938053097345
average:  72
Finish Rate:  0.6639825303671353
average:  73
Finish Rate:  0.6614386154678205
average:  74
Finish Rate:  0.6646570203644159
average:  75
Finish Rate:  0.6658644203770557


In [27]:
FinishRate(average_per_slot=120, machine_numbers=24)


Finish Rate:  0.523389159310288


In [10]:
for machine in [6,12,24]:
    for average in range(10,100,10):
        print("machine: ", machine, " average: ", average)
        FinishRate(average,machine)


machine:  6  average:  10
Finish Rate:  0.8121442125237192
machine:  6  average:  20
Finish Rate:  0.6793587174348698
machine:  6  average:  30
Finish Rate:  0.535857908847185
machine:  6  average:  40
Finish Rate:  0.45218476903870164
machine:  6  average:  50
Finish Rate:  0.36667995217218013
machine:  6  average:  60
Finish Rate:  0.30702179176755445
machine:  6  average:  70
Finish Rate:  0.27015032211882606
machine:  6  average:  80
Finish Rate:  0.2350432710397592
machine:  6  average:  90
Finish Rate:  0.21080903310677482
machine:  12  average:  10
Finish Rate:  0.8653100775193798
machine:  12  average:  20
Finish Rate:  0.8021148036253777
machine:  12  average:  30
Finish Rate:  0.7311827956989247
machine:  12  average:  40
Finish Rate:  0.6471898984897252
machine:  12  average:  50
Finish Rate:  0.5836909871244635
machine:  12  average:  60
Finish Rate:  0.5137344669718771
machine:  12  average:  70
Finish Rate:  0.46066282420749277
machine:  12  average:  80
Finish Rate:  0.4

In [28]:
import rllib_sac
import ray

ray.init()
rllib_sac.train_sac(30,6)


2023-09-30 11:12:30,610	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 
2023-09-30 11:12:31,803	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/a3c.py` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
2023-09-30 11:12:31,804	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/a3c.py` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
2023-09-30 11:12:31,870	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/maml/maml.py` has been deprecated. Use `rllib_contrib/maml/` instead. This will raise an error in the future!


{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 1, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'num_learner_workers': 0, 'num_gpus_per_learner_worker': 0, 'num_cpus_per_learner_worker': 1, 'local_gpu_idx': 0, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': 'VJS_6_30', 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'disable_env_checking': True, 'is_atari': None, 'auto_wrap_old_gym_envs': True, 'num_e

(SAC pid=818076) 2023-09-30 11:12:38,962	WARNING algorithm_config.py:643 -- Cannot create SACConfig from given `config_dict`! Property __stdout_file__ not supported.
(SAC pid=818076) 2023-09-30 11:12:38,979	INFO algorithm.py:536 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(SAC pid=818076) 2023-09-30 11:12:49,208	INFO trainable.py:173 -- Trainable.setup took 10.231 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(RolloutWorker pid=820210) 2023-09-30 11:13:10,966	WARNING env_runner_v2.py:154 -- More than 11180 observations in 11180 env steps for episode 814477653228105328 are buffered in the sampler. If this is more than you expected, check that that you set a horizon on your environment correctly and that it terminates at some point. Note: In multi-agent environments, `rollout_fragment_length` sets the batch size based on (across-agents) environ

2023-09-30 11:13:31,857	WARNING tune.py:192 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-09-30 11:13:42,053	INFO tune.py:1111 -- Total run time: 70.25 seconds (60.03 seconds for the tuning loop).
2023-09-30 11:13:42,054	WARNING tune.py:1126 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)


(RolloutWorker pid=820220) Traceback (most recent call last):
(RolloutWorker pid=820220)   File "python/ray/_raylet.pyx", line 1438, in ray._raylet.execute_task
(RolloutWorker pid=820220)   File "python/ray/_raylet.pyx", line 1378, in ray._raylet.execute_task.function_executor
(RolloutWorker pid=820220)   File "/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/_private/function_manager.py", line 724, in actor_method_executor
(RolloutWorker pid=820220)     return method(__ray_actor, *args, **kwargs)
(RolloutWorker pid=820220)   File "/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/util/tracing/tracing_helper.py", line 464, in _resume_span
(RolloutWorker pid=820220)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=820220)   File "/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/utils/actor_manager.py", line 176, in apply
(RolloutWorker pid=820220)     return func(self, *args, **kwargs)
(RolloutWorker pid=820220)   File "/home/yuan/ResMan/man/lib/python3.

In [1]:
import numpy as np
con = np.array([1])


In [3]:
con[0]


1

## Random

In [7]:
from environment import Environment
def get_env_continuous(average_per_slot=50, machine_num=12):
    para = Environment.VehicleJobSchedulingParameters(
        average_per_slot=average_per_slot, machine_numbers=machine_num
    )
    para.action_space_continuous = True
    env = Environment.VehicleJobSchedulingEnvACE(parameter=para)
    return env


In [10]:
finished = False
env = get_env_continuous(20,6)
env.reset()
reward_sums = {a: 0.0 for a in env.possible_agents}
for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()
    reward_sums[agent] += reward
    env.step(env.action_space(agent).sample())
    
print("rewards:")
print(reward_sums)
print("total reward:")
print(sum(reward_sums.values()))


rewards:
{'Machine_0': 1443952.3514621258, 'Machine_1': 3046659.2624404957, 'Machine_2': 2705154.313986431, 'Machine_3': 1563793.850536108, 'Machine_4': 3356951.536629907, 'Machine_5': 3638268.5951493443}
total reward:
15754779.910204412


In [3]:
env.finished_job/env.total_job


0.8440643863179075

In [9]:
env = get_env_continuous(20,6)
env.reset()
reward_sums = {a: 0.0 for a in env.possible_agents}
for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()
    reward_sums[agent] += reward
    env.step(1)
    
print("rewards:")
print(reward_sums)
print("total reward:")
print(sum(reward_sums.values()))


rewards:
{'Machine_0': 1675460.25, 'Machine_1': 4127737.0500000077, 'Machine_2': 3547264.9499999946, 'Machine_3': 1604004.75, 'Machine_4': 3371350.050000006, 'Machine_5': 3282498.4499999955}
total reward:
17608315.500000007


In [22]:
env.finished_job/env.total_job


0.5370744860128075

In [15]:
from ray.tune import register_env

import rllib_setup
def create_env(machine,jobs):
    env_name = "VJS"
    alg_name = "SAC"
    register_env(
        env_name,
        lambda config: rllib_setup.get_env_continuous(jobs, machine),
    )
    test_env = rllib_setup.get_env_continuous(jobs, machine)
    return test_env

test_env = create_env(12,60)

test_env.reset()

reward_sums = {a: 0.0 for a in test_env._agent_ids}


In [16]:
from ray.rllib.algorithms.algorithm import Algorithm
import ray

# ray.init()
algo = Algorithm.from_checkpoint("/home/yuan/ray_results/A3C/SAC_VJS_0c6b7_00000_0_2023-09-29_13-28-40/checkpoint_000700")


2023-10-04 15:50:32,128	INFO trainable.py:173 -- Trainable.setup took 11.342 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [24]:
algo.evaluate()


ValueError: Cannot evaluate w/o an evaluation worker set in the Trainer or w/o an env on the local worker!
Try one of the following:
1) Set `evaluation_interval` >= 0 to force creating a separate evaluation worker set.
2) Set `create_env_on_driver=True` to force the local (non-eval) worker to have an environment to evaluate on.

In [23]:
for agent in test_env.env.agent_iter():
    observation, reward, termination, truncation, info = test_env.env.last()
    print(test_env.env.last())
    reward_sums[agent] += reward
    if termination or truncation:
        action = None
    else:
        print(algo.get_policy(agent))
        policy = algo.get_policy(agent)
        batch_obs = {
            "obs": {
                "observation": observation,
                "action_mask": observation,
            }
        }
        batched_action, state_out, info = policy.compute_actions_from_input_dict(
            batch_obs
        )
        single_action = batched_action[0]
        action = single_action

    test_env.step(action)
    i += 1
    # env.render()

print("rewards:")
print(reward_sums)


(array([ 72., 144.,  72., 144.,  72., 144.,  72., 144.,  72., 144.,  72.,
       144.,  72., 144.,  72., 144.,  72., 144.,  72., 144.,  72., 144.,
        72., 144.,  72., 144.,  72., 144.,  72., 144.,  72., 144.,  72.,
       144.,  72., 144.,  72., 144.,  72., 144.,   0.,   0.,   1.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.]), 0, False, False, {})
SACTorchPolicy


RuntimeError: mat1 and mat2 shapes cannot be multiplied (73x2 and 73x256)

In [3]:
from environment import environment_jax
from environment import Environment
from pettingzoo.test import performance_benchmark



env = Environment.VehicleJobSchedulingEnvACE()
env.reset()
performance_benchmark(env)


Starting performance benchmark


11005.012010156304 turns per second
917.0843341796921 cycles per second
Finished performance benchmark
Starting performance benchmark
11391.133549607744 turns per second
949.2611291339787 cycles per second
Finished performance benchmark


In [4]:
import cProfile
env = environment_jax.VehicleJobSchedulingEnvACE()
env.reset()
cProfile.run("performance_benchmark(env)")


Starting performance benchmark
10990.468154029168 turns per second
915.8723461690973 cycles per second
Finished performance benchmark
Starting performance benchmark
7658.223707568317 turns per second
638.1853089640264 cycles per second
Finished performance benchmark
         3769624 function calls (3615914 primitive calls) in 5.004 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    36119    0.039    0.000    0.372    0.000 <__array_function__ internals>:177(all)
     8178    0.008    0.000    0.133    0.000 <__array_function__ internals>:177(argsort)
       33    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_1d)
       33    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(broadcast_arrays)
    38420    0.039    0.000    0.200    0.000 <__array_function__ internals>:177(concatenate)
      108    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(copyto)
    

In [2]:
import jax.numpy as np
np.max(np.array([24, 100]))


Array(100, dtype=int32)